# Supervised Modeling Pipeline for ICU Length of Stay Prediction

**Authors**: *Giuseppe Pitruzzella, Radvilė Rušaitė, Karlota Bochanaitė*

This project is situated within the field of supervised learning, aiming to predict a continuous clinical variable—**Length of Stay (LOS)** in the Intensive Care Unit—using regression models based on neural networks. The approach aligns with fundamental machine learning paradigms studied during the course, including probabilistic optimization techniques such as Maximum A Posteriori (MAP) estimation.

The analysis uses real-world data from **MIMIC-III** (Medical Information Mart for Intensive Care), a publicly available clinical database developed by the MIT Lab for Computational Physiology in collaboration with Beth Israel Deaconess Medical Center (Boston). The database contains de-identified health-related data associated with over 60,000 ICU admissions between 2001 and 2012, and has become a globally recognized benchmark for research in medical data science and critical care.

## Dataset Setup (Run Directly Below)

To facilitate reproducibility, the required `.csv` files from the MIMIC-III dataset are downloaded and decompressed **automatically** via the script cell provided just below. This script:

* Fetches the list of all available `.csv.gz` files from the educational [mirror site](https://www.dcc.fc.up.pt/~ines/MIMIC-III/)
* Downloads and decompresses each file
* Organizes them into the `data/raw/` folder

> Simply **run the code cell below** to fetch and prepare the full dataset.

```python
import os

# Absolute or relative path of the script
script_path = "../data/raw/download_mimic.sh"

# Check if it exists before executing
if os.path.exists(script_path):
    print("Executing download script...")
    !bash {script_path}
else:
    print("Script not found:", script_path)
```

In [ ]:
import subprocess
import sys
import importlib.util
import os

In [ ]:
# Absolute or relative path of the script
script_path = "../data/raw/download_mimic.sh"

# Check if it exists before executing
if os.path.exists(script_path):
    print("Executing download script...")
    !bash {script_path}
else:
    print("Script not found:", script_path)

## Environment Setup (Run Below)

Before proceeding with the analysis, make sure all required Python libraries are available. You can automatically install missing dependencies listed in `requirements.txt` by executing the following cell

In [ ]:
# Path to the requirements.txt file
req_file = "../requirements.txt"  # modify if necessary

def read_requirements(file_path):
    with open(file_path, "r") as f:
        return [line.strip().split("==")[0] for line in f if line.strip() and not line.startswith("#")]

def is_installed(package_name):
    return importlib.util.find_spec(package_name) is not None

def pip_install(package_name):
    print(f"Installing: {package_name}")
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

# Map exceptions between package name and importable module
module_map = {
    "scikit-learn": "sklearn",
    "ipython": "IPython"
}

# Load packages from requirements.txt
required_packages = read_requirements(req_file)

# Install only missing packages
for pkg in required_packages:
    module_name = module_map.get(pkg, pkg)
    if not is_installed(module_name):
        pip_install(pkg)
    else:
        print(f"Already installed: {pkg}")


## Initial Exploration

We begin by exploring the main tables in the MIMIC-III dataset. The goal of this first phase is to understand the structure of the database and identify relevant variables that may influence ICU length of stay. Key reference tables include:

* `D_ICD_DIAGNOSES.csv` – diagnosis codes (ICD9)
* `ICUSTAYS.csv` – ICU stays metadata
* `D_ITEMS.csv` – item IDs and descriptions for time-series events